In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
# 来源：An Estimation Method of Continuous Non-Invasive Arterial Blood Pressure Waveform Using Photoplethysmography: A U-Net Architecture-Based Approach
# 作者: Athaya
# 在自己的数据集上复现其模型

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
train_data = pd.read_csv(r'../input/blood-pressure-datasets/Train_Merge_Data.csv')
validation_data = pd.read_csv(r'../input/blood-pressure-datasets/Validation_Merge_Data.csv')

In [ ]:
train_data = shuffle(train_data)
validation_data = shuffle(validation_data)

In [ ]:
train_label_Sbp = train_data.iloc[:,610]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,611]
train_label_Dbp = train_label_Dbp.values

train_data = train_data.iloc[:,:600]
train_data = train_data.values

In [ ]:
validation_label_Sbp = validation_data.iloc[:,610]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,611]
validation_label_Dbp = validation_label_Dbp.values

validation_data = validation_data.iloc[:,:600]
validation_data = validation_data.values

In [ ]:
print("train_data_information:")
print(train_data.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)
print("validation_data_information:")
print(validation_data.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

In [ ]:
x=32
def create_model():
    inputs = keras.Input(shape=(600,1))

    conv1 = layers.Conv1D(x*2,3,padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation(tf.nn.relu)(conv1)

    conv1 = layers.Conv1D(x*2,3,padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation(tf.nn.relu)(conv1)

    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(x*4,3,padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation(tf.nn.relu)(conv2)

    conv2 = layers.Conv1D(x*4,3,padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation(tf.nn.relu)(conv2)

    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)
    
    
    conv3 = layers.Conv1D(x*8,3,padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation(tf.nn.relu)(conv3)

    conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation(tf.nn.relu)(conv3)

    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    
    conv4 = layers.Conv1D(x*16,3,padding='same')(pool3)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation(tf.nn.relu)(conv4)

    conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation(tf.nn.relu)(conv4)
    
    conv4 = layers.Dropout(0.5)(conv4)
    ####
    pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    

    
    conv5 = layers.Conv1D(x*32,3,padding='same')(pool4)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Activation(tf.nn.relu)(conv5)

    conv5 = layers.Conv1D(x*32,3,padding='same')(conv5)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Activation(tf.nn.relu)(conv5)
    
    conv5 = layers.Dropout(0.5)(conv5)
    
    
    ####
    conv5 = layers.UpSampling1D(size=2)(conv5)
    conv5 = layers.ZeroPadding1D((0,1))(conv5)
    
    conv5 = layers.Conv1D(x*16,2,padding='same')(conv5)
    
    up6 = layers.Concatenate(axis=2)([conv5,conv4])
    
    conv6 = layers.Conv1D(x*16,3,padding='same')(up6)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation(tf.nn.relu)(conv6)

    conv6 = layers.Conv1D(x*16,3,padding='same')(conv6)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation(tf.nn.relu)(conv6)
    
    ####
    conv6 = layers.UpSampling1D(size=2)(conv6)
    
    conv6 = layers.Conv1D(x*8,2,padding='same')(conv6)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation(tf.nn.relu)(conv6)
    
    up7 = layers.Concatenate(axis=2)([conv6,conv3])

    conv7 = layers.Conv1D(x*8,3,padding='same')(up7)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Activation(tf.nn.relu)(conv7)

    conv7 = layers.Conv1D(x*8,3,padding='same')(conv7)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Activation(tf.nn.relu)(conv7)
    
    ####
    
    conv7 = layers.UpSampling1D(size=2)(conv7)
    
    conv7 = layers.Conv1D(x*4,2,padding='same')(conv7)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Activation(tf.nn.relu)(conv7)
    
    up8 = layers.Concatenate(axis=2)([conv7,conv2])

    conv8 = layers.Conv1D(x*4,3,padding='same')(up8)
    conv8 = layers.BatchNormalization()(conv8)
    conv8 = layers.Activation(tf.nn.relu)(conv8)

    conv8 = layers.Conv1D(x*4,3,padding='same')(conv8)
    conv8 = layers.BatchNormalization()(conv8)
    conv8 = layers.Activation(tf.nn.relu)(conv8)
    
    ####
    
    conv8 = layers.UpSampling1D(size=2)(conv8)
    
    conv8 = layers.Conv1D(x*2,2,padding='same')(conv8)
    conv8 = layers.BatchNormalization()(conv8)
    conv8 = layers.Activation(tf.nn.relu)(conv8)
    

    up9 = layers.Concatenate(axis=2)([conv8,conv1])

    conv9 = layers.Conv1D(x*2,3,padding='same')(up9)
    conv9 = layers.BatchNormalization()(conv9)
    conv9 = layers.Activation(tf.nn.relu)(conv9)

    conv9 = layers.Conv1D(x*2,3,padding='same')(conv9)
    conv9 = layers.BatchNormalization()(conv9)
    conv9 = layers.Activation(tf.nn.relu)(conv9)


    com_layer = layers.GlobalAveragePooling1D()(conv9)
    
    com_layer = layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.4))(com_layer)
    
    outputs_sbp = layers.Dense(1,name='Sbp')(com_layer)
    outputs_dbp = layers.Dense(1,name='Dbp')(com_layer)

    model = keras.Model(inputs=inputs,outputs=[outputs_sbp,outputs_dbp])
    
    return model

In [ ]:
"""自定义评价指标模块"""
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction.h5'
Save_epochs = 10 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

In [ ]:
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph.png', show_shapes=True, show_layer_names=False)

In [ ]:
history = model.fit(train_data,{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=(validation_data,{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
with open(r'./last_model.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)